In [56]:
from __future__ import absolute_import, division, print_function

import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error, accuracy_score

import torch
# from torch.distributions import constraints

import pyro
import pyro.distributions as dist
from pyro.distributions import Normal, Uniform, Delta
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, JitTrace_ELBO, TracePredictive
from pyro.contrib.autoguide import AutoMultivariateNormal
from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import NUTS
from pyro.infer.mcmc.util import diagnostics
import pyro.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from pyro.ops.stats import waic
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from copy import deepcopy
from imblearn.under_sampling import RandomUnderSampler


pyro.set_rng_seed(1)
assert pyro.__version__.startswith('0.4.1')

In [55]:
!pip install imblearn

In [6]:
%matplotlib inline
logging.basicConfig(format='%(message)s', level=logging.INFO)
# Enable validation checks
pyro.enable_validation(True)
smoke_test = ('CI' in os.environ)
pyro.set_rng_seed(1)

In [57]:
data = pd.read_csv('data/crap.csv')

In [58]:
data['target'] = data.apply(lambda row : row['FELONY'] + row['MISDEMEANOR'] + row['VIOLATION'], axis=1)

In [59]:
data['target'] = data['target'].astype(int)

In [72]:
def get_data(data, features, target):
    df_X = data[features]
    df_y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2)
    # Train Data
    X_np_train = normalize(np.array(X_train))
    y_np_train = np.array(y_train)
    
    rus = RandomUnderSampler(random_state=42)
    X_np_train, y_np_train = rus.fit_resample(X_np_train, y_np_train)
    
    print(y_np_train.sum(), len(y_np_train)-y_np_train.sum())

    X_nuts_train = torch.from_numpy(X_np_train).type(torch.float32)
    y_nuts_train = torch.from_numpy(y_np_train).type(torch.float32)

   
    population =  torch.from_numpy(np.array(X_train['TotalPop'])).type(torch.float32)

    #Test Data
    X_np_test = normalize(np.array(X_test))
    y_np_test = np.array(y_test)

    X_nuts_test = torch.from_numpy(X_np_test).type(torch.float32)
    y_nuts_test = torch.from_numpy(y_np_test).type(torch.float32)

    test_population =  torch.from_numpy(np.array(X_test['TotalPop'])).type(torch.float32)
    return X_nuts_train, y_nuts_train, population, X_nuts_test, y_nuts_test, test_population


In [73]:
bool_split = 3
offset = 3

In [74]:
def sep_data(row):
    if row['target'] >= bool_split + offset :
        return 1
    elif row['target'] <= bool_split - offset :
        return 0
    else:
        return float('nan')

In [75]:
data.columns

Index(['Unnamed: 0', 'month', 'CensusTract', 'TotalPop', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'FELONY', 'MISDEMEANOR',
       'VIOLATION', 'restaurants', 'bar', 'park', 'subway_station', 'target'],
      dtype='object')

In [76]:
features = ['TotalPop', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment','restaurants', 'bar', 'park', 'subway_station']
target = 'binary_target'            

In [ ]:
# for bool_split in range(2,10):
#     for offset in range(1,5):

bool_split = 5
offset = 2

for bool_split in range(2,20):
    for offset in range(1,10):
        if bool_split - offset <= 0:
            continue
        print(bool_split, offset)
        data_new = deepcopy(data)
        data_new['binary_target'] = data_new.apply(sep_data, axis=1)
        data_new = data_new.dropna()
#         print(len(data_new[data_new['binary_target']==0]), len(data_new[data_new['binary_target']==1]))
        if (len(data_new[data_new['binary_target']==0]) == 0) or (len(data_new[data_new['binary_target']==1]) == 0):
            print('-'*30)
            continue
        X_nuts_train, y_nuts_train, population, X_nuts_test, y_nuts_test, test_population = get_data(data_new, features, target)
        reg = LogisticRegression(C=0.8).fit(X_nuts_train, y_nuts_train)
        print('Train Score : ',reg.score(X_nuts_train, y_nuts_train))
        print('Test Score :', reg.score(X_nuts_test, y_nuts_test))
        print('-'*30)

        

2 1
3513.0 3513.0
Train Score :  0.6827497865072587
Test Score : 0.6382252559726962
------------------------------
3 1


/Users/raksh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


6588.0 6588.0
Train Score :  0.6955828779599271
Test Score : 0.6735807860262009
------------------------------
3 2


/Users/raksh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


3507.0 3507.0
Train Score :  0.7392358140861135
Test Score : 0.7255389718076285
------------------------------
4 1


...

/Users/raksh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


146.0 146.0
Train Score :  0.7123287671232876
Test Score : 0.7390130125214829
------------------------------
16 6


/Users/raksh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


114.0 114.0
Train Score :  0.7412280701754386
Test Score : 0.7941912869303956
------------------------------
16 7


/Users/raksh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
